In [1]:
import sys
import os
import cv2
# from google.colab.patches import cv2_imshow
import mediapipe as mp
import numpy as np
import pandas as ps
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from skimage import color

from scipy.spatial import distance as dist
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from numpy import *
from matplotlib.colors import ListedColormap
import tensorflow as tf
from tensorflow import keras

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
def getContours(img, imgContour):
    # find all the contours from the B&W image
    contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # needed to filter only our contours of interest
    finalContours = []

    # for each contour found
    for cnt in contours:
        # cv2.drawContours(imgContour, cnt, -1, (255, 255, 0), 1)
        # find its area in pixel
        area = cv2.contourArea(cnt)
        # print("Detected Contour with Area: ", area)

        perimeter = cv2.arcLength(cnt, True)

        # smaller epsilon -> more vertices detected [= more precision]
        epsilon = 0.002 * perimeter
        # check how many vertices
        approx = cv2.approxPolyDP(cnt, epsilon, True)
        # print(len(approx))

        finalContours.append([len(approx), area, approx, cnt])

    # print("---\nFinal number of External Contours: ", len(finalContours))
    # sorting in descending order depending on the area
    finalContours = sorted(finalContours, key=lambda x: x[1], reverse=True)

    # drawing contours for the final objects
    # for con in finalContours:
    #    cv2.drawContours(imgContour, con[3], -1, (0, 0, 255), 3)

    return imgContour, finalContours

In [4]:
def getGeometricFeatures(path, mouth):
    mpFaceMesh = mp.solutions.face_mesh
    faceMesh = mpFaceMesh.FaceMesh(max_num_faces=1)
    
#     print("PATH: ",path)
    img = cv2.imread(path)
    b,g,r = cv2.split(img)           # get b, g, r
    rgb_img1 = cv2.merge([r,g,b])
    
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = faceMesh.process(imgRGB)

    width = 0
    heigh = 0
    mar = 0
    
    p_up_inner = 0
    p_down_inner = 0
    p_right_inner = 0
    p_left_inner = 0
    top_mouth = 0
    p_up_outer = 0
    p_down_outer = 0
    p_left_outer = 0
    
    a_up = 0 
    a_down = 0
    b_up = 0 
    b_down = 0
    d_up = 0
    d_down = 0
    e_up = 0
    e_down = 0

    if results.multi_face_landmarks:
        for faceLns in results.multi_face_landmarks:
            for id_, lm in enumerate(faceLns.landmark):
                ih, iw, ic = img.shape
                x, y , z = int(lm.x*iw), int(lm.y*ih), int(lm.z*ic)

                if id_ == 13: 
                    p_up_inner = [x, y]
                    
                elif id_ == 17: 
                    p_down_outer = [x, y]
                
                elif id_ == 61: 
                    p_left_outer = [x, y]
                    
                elif id_ == 291: 
                    p_right_outer = [x, y]
                    
                elif id_ == 0: 
                    p_up_outer = [x, y]
                    
                elif id_ == 14: 
                    p_down_inner = [x, y] 
                    
                elif id_ == 78: 
                    p_left_inner = [x, y]
                    
                elif id_ == 308: 
                    p_right_inner = [x, y]
                    
                elif id_ == 164: 
                    top_mouth = [x, y]
                    

                    
# #                   outer left cupid                    
#                 elif id_ == 37:
#                     a_up = [x, y]
                    
#                 elif id_ == 84:
#                     a_down = [x, y]
    
# #                 outer right cupid
#                 elif id_ == 267:
#                     b_up = [x, y]
    
#                 elif id_ == 314:
#                     b_down = [x, y]
    

    
     
                    
    if top_mouth != 0:
        width = (p_right_inner[0] - p_left_inner[0])
        heigh = (p_down_inner[1] - p_up_inner[1]) 
        mouth = rgb_img1[p_up_outer[1]:p_down_outer[1], p_left_outer[0]:p_right_outer[0]]

#         A = dist.euclidean(a_up, a_down)
#         B = dist.euclidean(b_up, b_down)

        
        
        # compute the euclidean distance between the horizontal
        # mouth landmark (x, y)-coordinates
#         F = dist.euclidean(p_left_outer, p_right_outer)
#         G = (A + B)
        
        # compute the mouth aspect ratio
#         mar = G / (2.0 * F)
        
#         print(mar)
#         width, heigh,
        
    return  mouth,width, heigh

In [5]:
def getDarkArea(mouthImage):
    # transform the image into gray scale
    
    grayImage = cv2.cvtColor(mouthImage, cv2.COLOR_BGR2GRAY)
    slicePixelArea = 0
    
    # transform the gray scale image into black and white
    (thresh, blackAndWhiteImage) = cv2.threshold(grayImage, 20, 255, cv2.THRESH_BINARY)

    # for the edges of the dark area
    imgCanny = cv2.Canny(blackAndWhiteImage, 255, 195)
    kernel = np.ones((2, 2))
    imgDil = cv2.dilate(imgCanny, kernel, iterations=3)
    imgThre = cv2.erode(imgDil, kernel, iterations=3)
    
    imgFinalContours, finalContours = getContours(imgThre, mouthImage)
#     imgplot = plt.imshow(imgThre) 
#     plt.show()
#     imgplot2 = plt.imshow(imgFinalContours) 
#     plt.show()
#     cv2.imshow('', imgFinalContours)
    
#     cv2.waitKey(0)

    
    if len(finalContours) != 0:
        slicePixelArea = finalContours[0][1]
#         print("Entire Slice Area in pixel", slicePixelArea)
    return slicePixelArea

In [6]:
def getFeatures(path, i, personne):
    features = []#ps.DataFrame()
    temp_height = 0
    temp_width = 0
    temp_dark  = 0
    temp_mar = 0
    ctr = 0
    
    #features.append(personne)
#     print("Get Feature path: "+ path)
    for imgName in os.listdir(path):
        
        image_path = path + "/" + imgName
        
#          
        mouth, width, heigh, = getGeometricFeatures(image_path, 0)
        
        features.append(width)
        features.append(heigh)
#         features.append(mar)
#         features.append(getDarkArea(mouth))
#         dark = getDarkArea(mouth)
#         print('\nWIDTH: ', width, 'HEIGHT: ', heigh, 'DARK AREA:', dark)
        temp_height = heigh
        temp_width = width
#         temp_dark = dark
#         temp_mar= mar

        ctr +=1
    
#         print("width: ", width, "height: ", heigh,"mar: ", mar)
    print(features)
    if ((len(features))/2) >18:
        exceed_length = True
        f_geoFeatures = list(zip(features[::2], features[1::2]))
        n_geo = f_geoFeatures[::2]
#         three = list(zip(*[iter(features)]*3))
#         n_geo = features[::2]
        features =  np.array(n_geo).flatten()
        features = list(features)

    length_fea = int(len(features)/2)
#     print("length: ",length_fea)
    if length_fea <18:
        num_fea_append = 18 - length_fea
#         print(temp_height, temp_width, "CTR: ", ctr)
        
        for x in range(num_fea_append):
            features.append(temp_width)
            features.append(temp_height)
#             features.append(temp_dark)
#             features.append(temp_mar)

    print("Total Syllable: ",(len(features))/2)
    features.append(i)
    
#     print("features shape : ", len(features))
    
    return features

In [7]:
def syllabeFeatures(path, i):
    syllabeFeatures = []
    #syllabeFeatures = np.array(syllabeFeatures)
    for sd in sorted(os.listdir(path)):
        print("\n\nVideo Name: " + sd)
        sub_sub_dir = path + "/" + sd
#         print("i: "+i)

        #syllabeFeatures = np.append(syllabeFeatures, getFeatures(sub_sub_dir, i))
        syllabeFeatures.append(getFeatures(sub_sub_dir, i, sd))
    #yllabeFeatures = np.array(syllabeFeatures)
#         print("\nsyllabeFeatures: ", syllabeFeatures)
    print("data shape : ", np.array(syllabeFeatures).shape)
    
        #if sd == "M2U02788" : break

    return syllabeFeatures

In [8]:
path = 'C:/Users/sokiy/Desktop/Thesis/imageframeslm/augment/x/'
# train_path = "../input/voyelleface/voyelles/train/"
i = 0
geoFeatures = []
for d in os.listdir(path):
    path = path +d
    print("path",path)
    print("D: "+ d)
    syl_features = syllabeFeatures(path, d)
    geoFeatures.append(syl_features)
#     print("geo: " ,geoFeatures)
    i += 1


path C:/Users/sokiy/Desktop/Thesis/imageframeslm/augment/x/wi
D: wi


Video Name: wi-61_a0
[113, 3, 110, 2, 107, 3, 107, 2, 108, 4, 102, 4, 103, 3, 105, 3, 107, 3, 107, 3, 105, 3, 109, 2, 110, 4, 114, 7, 111, 8, 112, 7, 115, 7, 110, 7, 112, 9, 111, 6, 110, 6, 110, 7, 113, 6, 119, 4]
Total Syllable:  18.0


Video Name: wi-61_a1
[112, 3, 114, 1, 109, 2, 107, 2, 109, 5, 110, 5, 104, 3, 105, 2, 107, 2, 108, 2, 113, 3, 116, 2, 112, 4, 115, 7, 116, 7, 119, 12, 121, 12, 109, 9, 110, 7, 109, 5, 117, 10, 114, 6, 112, 5, 119, 3]
Total Syllable:  18.0


Video Name: wi-61_a2
[113, 2, 112, 2, 107, 2, 107, 3, 106, 4, 105, 4, 106, 2, 108, 2, 107, 3, 107, 2, 107, 3, 109, 2, 112, 5, 108, 6, 114, 11, 116, 11, 117, 10, 109, 7, 111, 7, 109, 8, 110, 6, 108, 7, 111, 5, 118, 3]
Total Syllable:  18.0


Video Name: wi-62_a0
[104, 3, 104, 2, 106, 3, 103, 3, 100, 5, 97, 4, 99, 5, 100, 5, 97, 5, 94, 7, 99, 6, 101, 6, 96, 6, 94, 7, 93, 8, 95, 7, 97, 8, 111, 5, 121, 8, 119, 4, 124, 11, 118, 4]
Total Syllable:  18.0

[106, 4, 105, 8, 101, 10, 101, 11, 99, 11, 102, 11, 101, 9, 100, 9, 100, 8, 100, 8, 100, 7, 98, 7, 101, 7, 101, 6, 101, 6, 102, 6, 103, 6, 102, 6, 106, 9, 105, 13, 109, 17, 107, 14, 112, 16]
Total Syllable:  18.0


Video Name: wi-73_a1
[110, 6, 105, 7, 106, 11, 100, 10, 100, 9, 105, 11, 101, 8, 101, 9, 100, 9, 100, 9, 100, 7, 99, 7, 103, 8, 101, 8, 102, 8, 99, 7, 104, 7, 103, 7, 106, 10, 105, 11, 108, 16, 106, 12, 109, 19]
Total Syllable:  18.0


Video Name: wi-73_a2
[108, 5, 106, 7, 100, 10, 103, 12, 100, 10, 102, 14, 101, 9, 101, 9, 98, 9, 99, 8, 99, 6, 100, 6, 99, 7, 98, 7, 102, 7, 101, 7, 103, 6, 104, 6, 103, 10, 104, 14, 106, 14, 106, 11, 111, 18]
Total Syllable:  18.0


Video Name: wi-74_a0
[105, 5, 105, 7, 100, 10, 101, 9, 101, 11, 101, 15, 101, 10, 101, 8, 99, 8, 98, 8, 97, 7, 99, 7, 99, 7, 99, 7, 99, 7, 104, 7, 101, 6, 103, 7, 105, 9, 105, 14, 105, 13, 109, 14, 111, 15]
Total Syllable:  18.0


Video Name: wi-74_a1
[107, 5, 106, 7, 105, 12, 100, 10, 101, 10, 105, 11, 105, 11, 1

[105, 6, 101, 7, 102, 7, 103, 9, 107, 15, 112, 17, 111, 15, 112, 11, 114, 10, 116, 6, 119, 8]
Total Syllable:  18.0


Video Name: wi-86_a1
[100, 7, 105, 8, 103, 6, 105, 8, 107, 12, 114, 16, 110, 14, 110, 9, 116, 9, 116, 6, 118, 6]
Total Syllable:  18.0


Video Name: wi-86_a2
[104, 6, 104, 6, 105, 5, 103, 7, 105, 10, 113, 15, 113, 16, 110, 10, 113, 8, 115, 6, 117, 5]
Total Syllable:  18.0


Video Name: wi-87_a0
[96, 6, 93, 6, 97, 5, 99, 5, 94, 6, 92, 6, 93, 5, 95, 4, 92, 5, 93, 5, 89, 5, 95, 6, 92, 6, 97, 5, 98, 7, 97, 4, 97, 6, 99, 6, 95, 5]
Total Syllable:  18.0


Video Name: wi-87_a1
[99, 7, 100, 5, 102, 7, 96, 5, 94, 6, 96, 5, 91, 6, 94, 5, 94, 5, 97, 5, 94, 5, 94, 5, 95, 6, 100, 5, 95, 7, 97, 5, 104, 7, 100, 6, 105, 6]
Total Syllable:  18.0


Video Name: wi-87_a2
[97, 6, 98, 5, 97, 5, 95, 6, 92, 7, 92, 6, 92, 6, 94, 5, 94, 4, 94, 5, 93, 5, 93, 6, 96, 6, 99, 6, 94, 7, 100, 5, 103, 7, 98, 5, 97, 6]
Total Syllable:  18.0


Video Name: wi-88_a0
[96, 4, 94, 5, 92, 5, 94, 5, 94, 5, 94, 6

Append

In [9]:
df_final = ps.DataFrame()
# ,'darkArea'
columnsNames = ['width', 'height']
for feature in geoFeatures:
    for fea in feature:
        columns_ = []
#         change last num for features
        nb = (int((np.array(fea).shape[0]-1)/2))
        #columns_.append('personne')
        temp = [name + str(i) for i in range(nb) for name in columnsNames]
        columns_.extend(temp)
        columns_.append('i')
        df = ps.DataFrame(data = [fea], columns = columns_)
        df_final = df_final.append(df, 1)
        
        

In [10]:
df_final

,width0,height0,width1,height1,width2,height2,width3,height3,width4,height4,...,height13,width14,height14,width15,height15,width16,height16,width17,height17,i
0,113,3,107,3,108,4,103,3,107,3,...,4,119,4,119,4,119,4,119,4,wi
1,112,3,109,2,109,5,104,3,107,2,...,3,119,3,119,3,119,3,119,3,wi
2,113,2,107,2,106,4,106,2,107,3,...,3,118,3,118,3,118,3,118,3,wi
3,104,3,106,3,100,5,99,5,97,5,...,4,118,4,118,4,118,4,118,4,wi
4,106,3,106,2,100,5,104,4,96,6,...,5,118,5,118,5,118,5,118,5,wi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,88,5,98,5,94,4,102,6,108,15,...,17,111,17,111,17,111,17,111,17,wi
89,89,6,96,5,94,5,101,7,104,10,...,19,109,19,109,19,109,19,109,19,wi
90,101,3,100,5,94,3,93,3,97,4,...,3,110,3,110,3,110,3,110,3,wi
91,108,4,101,4,96,3,96,5,103,4,...,3,115,3,115,3,115,3,115,3,wi


In [11]:
df_final.to_csv("C:/Users/sokiy/Desktop/Thesis/csv/filter/augment/wi_aug2.csv", encoding='utf-8', index=False)